In [1]:
from google.cloud import storage

ModuleNotFoundError: No module named 'google'

In [ ]:
import os
from google.cloud import storage
from google.api_core import exceptions

def upload_invoice_to_gcs(bucket_name, source_file_path, destination_blob_name=None):
    """
    Uploads an invoice image to Google Cloud Storage.

    Args:
        bucket_name (str): The name of your GCS bucket.
        source_file_path (str): The local path to the invoice image file.
        destination_blob_name (str, optional): The name for the file in GCS.
                                               If None, uses the filename from source_file_path.

    Returns:
        str: The full GCS URI (gs://) of the uploaded file, or None if upload failed.
    """
    try:
        # Initialize the client using credentials from GOOGLE_APPLICATION_CREDENTIALS
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)

        # Determine the blob name (filename in GCS)
        if destination_blob_name is None:
            destination_blob_name = os.path.basename(source_file_path)

        blob = bucket.blob(destination_blob_name)

        # Upload the file
        print(f"Uploading {source_file_path} to gs://{bucket_name}/{destination_blob_name}...")
        blob.upload_from_filename(source_file_path)
        print(f"Upload complete.")

        # Return the GCS URI
        gcs_uri = f"gs://{bucket_name}/{destination_blob_name}"
        return gcs_uri

    except FileNotFoundError:
        print(f"Error: The file {source_file_path} was not found.")
        return None
    except exceptions.GoogleAPIError as e:
        print(f"Google Cloud Storage API error during upload: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during upload: {e}")
        return None

def make_gcs_object_public(bucket_name, blob_name):
    """
    Makes a GCS object publicly readable.

    Args:
        bucket_name (str): The name of your GCS bucket.
        blob_name (str): The name of the blob (file) in the bucket.

    Returns:
        str: The public URL of the object, or None if making public failed.
    """
    try:
        # Initialize the client
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # Make the blob publicly readable
        print(f"Making gs://{bucket_name}/{blob_name} publicly accessible...")
        blob.make_public()
        print("Object is now publicly accessible.")

        # Return the public URL
        public_url = blob.public_url
        print(f"Public URL: {public_url}")
        return public_url

    except exceptions.GoogleAPIError as e:
        print(f"Google Cloud Storage API error while making object public: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while making object public: {e}")
        return None


# --- Example Usage ---
if __name__ == "__main__":
    # --- REPLACE THESE VALUES ---
    YOUR_BUCKET_NAME = "swarm-invoices"
    LOCAL_INVOICE_IMAGE_PATH = "path/to/your/local/invoice.jpg" # e.g., "invoices/invoice_12345.jpg"
    # Optional: Specify a custom name for the file in GCS
    # DESTINATION_BLOB_NAME = "invoices/uploaded_invoice.jpg"
    DESTINATION_BLOB_NAME = None # Use the local filename
    # --------------------------

    # Step 1: Upload the image
    gcs_uri = upload_invoice_to_gcs(
        bucket_name=YOUR_BUCKET_NAME,
        source_file_path=LOCAL_INVOICE_IMAGE_PATH,
        destination_blob_name=DESTINATION_BLOB_NAME
    )

    if gcs_uri:
        print(f"Image uploaded successfully to GCS: {gcs_uri}")

        # Extract the blob name from the GCS URI for the next step
        # gs://bucket_name/blob_name -> blob_name
        blob_name = gcs_uri[len(f"gs://{YOUR_BUCKET_NAME}/"):]

        # Step 2: Make the uploaded image publicly accessible
        public_url = make_gcs_object_public(YOUR_BUCKET_NAME, blob_name)

        if public_url:
            print(f"\nSuccess! The public URL for your invoice image is:")
            print(public_url)
            print("\nYou can use this URL for integration with systems like Google Wallet,")
            print("provided it meets their specific data requirements (e.g., format, content).")
        else:
            print("Failed to make the image publicly accessible.")
    else:
        print("Failed to upload the image.")
